# <font color="#49699E" size=40>Influence, Inequality, and Power in Social Networks</font>
# LEARNING OBJECTIVES
# LEARNING MATERIALS
# INTRODUCTION


## Package Imports

In [ ]:
import networkx as nx
import pandas as pd
pd.set_option("display.notebook_repr_html", False)
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from dcss.plotting import format_axes_commas, custom_seaborn
from dcss.networks import *
custom_seaborn()

import collections
import random

## Data


In [ ]:
contact_diaries = pd.read_csv("../data/SocioPatterns/Contact-diaries-network_data_2013.csv", sep=' ')

G = nx.from_pandas_edgelist(contact_diaries, 'i', 'j', create_using=nx.Graph())
G.name = 'Reported Contacts (Diary Data)'
print(nx.info(G))

# CENTRALITY MEASURES: THE BIG PICTURE


# SHORTEST PATHS AND NETWORK FLOW

## Shortest Paths / Geodesics 


In [ ]:
layout = nx.kamada_kawai_layout(G)

In [ ]:
path_a, es_a = get_shortest_paths(G, 173, 48) 
print(path_a)

In [ ]:
plot_path(G, layout, path_a, es_a)

### Multiple Shortest Paths


In [ ]:
sps = [path for path in nx.all_shortest_paths(G, source=173, target=48)]
path_nodes = set([item for sublist in sps for item in sublist])

for path in sps:
    print(path)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

nx.draw_networkx_nodes(G, pos=layout, node_size=200, node_color='#32363A')

nx.draw_networkx_edges(G,
                       pos=layout,
                       edge_color='darkgray',
                       width=1)

# the paths!

nx.draw_networkx_nodes(G,
                       pos=layout,
                       node_size=200,
                       node_color='crimson',
                       nodelist=path_nodes)

for p in sps:
    edges = set(zip(p, p[1:]))
    nx.draw_networkx_edges(G,
                           pos=layout,
                           edgelist=edges,
                           edge_color='crimson',
                           width=4)

plt.axis('off')
plt.show()

In [ ]:
nx.shortest_path_length(G, source=173, target=48) 

In [ ]:
np.round(nx.average_shortest_path_length(G), 2)

# BETWEENNESS CENTRALITY, TWO WAYS


In [ ]:
nx.draw(nx.barbell_graph(5,1), node_size=300, node_color='#32363A')

In [ ]:
sp_bet = pd.Series(nx.betweenness_centrality(G))

ax = sns.histplot(sp_bet, kde=True)
ax.set(xlabel='Shortest path betweenness centrality', 
      ylabel='Number of nodes')
sns.despine()
plt.show()

In [ ]:
cf_bet = pd.Series(nx.current_flow_betweenness_centrality(G))
betweenness = pd.concat([sp_bet, cf_bet], axis=1)
betweenness.columns = ['Shortest Path Betweenness', 'Current Flow Betweenness']

In [ ]:
sns.jointplot(data=betweenness,
              x='Shortest Path Betweenness',
              y='Current Flow Betweenness',
              alpha=.7)
plt.show()

# POPULARITY, POWER, AND INFLUENCE


In [ ]:
reported_friendships = pd.read_csv("../data/SocioPatterns/Friendship-network_data_2013.csv",
                                   sep=' ')

G_friendships = nx.from_pandas_edgelist(reported_friendships,
                                        'i', 'j', create_using=nx.DiGraph())

G_friendships.name = 'Reported Friendships'
print(nx.info(G_friendships)) 

In [ ]:
layout = nx.nx_pydot.graphviz_layout(G_friendships)

## Degree, Degree Centrality, and Connection Inequality


In [ ]:
def visualize_digraph(network, layout, node_size=50, title=''):
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_title(title, fontsize=16)
    nx.draw_networkx_nodes(network,
                       pos=layout,
                       node_size=node_size,
                       node_color='#32363A')
    nx.draw_networkx_edges(network,
                       pos=layout,
                       edge_color='#98989C',
                       arrowsize=5,
                       width=1)
    plt.axis('off')
    plt.show()

In [ ]:
in_degree = dict(G_friendships.in_degree())
out_degree = dict(G_friendships.out_degree())

In [ ]:
sized_by_indegree = [v * 20 for v in in_degree.values()]
sized_by_outdegree = [v * 20 for v in out_degree.values()]

In [ ]:
visualize_digraph(G_friendships, layout, sized_by_indegree)

In [ ]:
visualize_digraph(G_friendships, layout, sized_by_outdegree)

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(x=in_degree, y=out_degree, alpha = 0.2)
sns.despine()
ax.set(xlabel='Indegree',
       ylabel='Outdegree')
plt.show()

### Connection Inequality


In [ ]:
degree_sequence = sorted([d for n, d in G_friendships.degree()], reverse=True)  # degree sequence
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
plt.bar(deg, cnt, width=0.80, color="#32363A")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel('Degree')
ax.set_ylabel('Number of nodes')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

ax.loglog(degree_sequence, 
          'black', 
          marker='o', 
          markersize=3)

plt.title("Degree Rank Plot")
plt.ylabel("Degree")
plt.xlabel("Rank")

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

## Eigenvector Centrality


In [ ]:
def plot_nodes(which_network, which_nodes, what_color, where):
    if type(which_nodes) is list:
        nx.draw_networkx_nodes(which_network, 
                               pos=where, 
                               node_size=100, 
                               node_color=what_color,
                               nodelist=which_nodes) 
    else:
        nx.draw_networkx_nodes(which_network, 
                               pos=where, 
                               node_size=100, 
                               node_color=what_color,
                               nodelist=[which_nodes]) 

In [ ]:
alters = nx.ego_graph(G_friendships, 1519, radius=1, undirected=True)
alters_2 = nx.ego_graph(G_friendships, 1519, radius=2, undirected=True)

fig, ax = plt.subplots(figsize=(12, 8))
plot_nodes(G_friendships, list(G_friendships.nodes()), 'lightgray', layout)
plot_nodes(G_friendships, list(alters_2.nodes()), 'gray', layout)
plot_nodes(G_friendships, list(alters.nodes()), 'black', layout)
plot_nodes(G_friendships, 1519, 'crimson', layout)

nx.draw_networkx_edges(G_friendships, pos=layout, edge_color='lightgray',
                       arrowsize=3,
                       width=1)

plt.axis('off')
plt.show()

In [ ]:
alters = nx.ego_graph(G_friendships, 196, radius = 1, undirected = True)
alters_2 = nx.ego_graph(G_friendships, 196, radius = 2, undirected = True)

fig, ax = plt.subplots(figsize=(12,8))
plot_nodes(G_friendships, list(G_friendships.nodes()), 'lightgray', layout)
plot_nodes(G_friendships, list(alters_2.nodes()), 'gray', layout)
plot_nodes(G_friendships, list(alters.nodes()), 'black', layout)
plot_nodes(G_friendships, 196, 'crimson', layout)

nx.draw_networkx_edges(G_friendships, pos=layout, edge_color='lightgray',
                       arrowsize=3,
                       width=1)
plt.axis('off')
plt.show()

### Computing Eigenvector Centrality


In [ ]:
dn = pd.Series(dict(nx.degree(G_friendships)))
ec = pd.Series(nx.eigenvector_centrality(G_friendships))

fig, ax = plt.subplots()
sns.scatterplot(x=dn, y=ec, alpha=.6)
ax.set(xlabel='Degree', ylabel='Eigenvector centrality')
sns.despine()
plt.show()

## Bonacich Power Centrality


In [ ]:
G = nx.from_pandas_edgelist(contact_diaries, 'i', 'j', create_using=nx.Graph())
G.name = 'Reported Contacts (Diary Data)'

import os
nx.write_edgelist(G, "intermediary.csv", delimiter=',', data=False)
os.system('Rscript power_centrality.R')

In [ ]:
power = pd.read_csv('boncent.csv', index_col=0)
power = power.round(4)

In [ ]:
layout = nx.nx_pydot.graphviz_layout(G)

def size_by_beta_centrality(network, beta_value, title):
    size = power[beta_value].tolist()
    scale = abs(min(size))+0.0001
    size = [(s+scale)*1000 for s in size]
    visualize_digraph(G, layout, size, title)

In [ ]:
size_by_beta_centrality(G, 'b0', title=r'$\beta$ = 0 (i.e., Degree Centrality)')

In [ ]:
size_by_beta_centrality(G, 'b_neg.4', title=r'$\beta$ = -0.4')

In [ ]:
size_by_beta_centrality(G, 'b.4', title=r'$\beta$ = 0.4')

# CONCLUSION
## Key Points 


# Chapter Appendix
## Optional: Formulas for Centrality Measures Covered in this Chapter
### Shortest Path Betweenness
### Current Flow Betweenness
### Eigenvector Centrality
